In [1]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import numpy as np
from scipy.interpolate import make_interp_spline
from sklearn.model_selection import train_test_split
from gpytorch.kernels import Kernel
import pickle
from sklearn.preprocessing import StandardScaler
from gpytorch.means import Mean
from sklearn.metrics import mean_squared_error
from gpytorch.kernels import Kernel, RBFKernel,ScaleKernel
from gpytorch.constraints import Interval
from gpytorch.settings import cholesky_jitter
from gpytorch.models import ApproximateGP
from gpytorch.variational import CholeskyVariationalDistribution, VariationalStrategy
import torch.nn as nn
import logging
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Global variables and configuration

In [2]:
# 创建一个logger
logger = logging.getLogger('my_logger')
logger.setLevel(logging.DEBUG)

# 创建文件处理器，写入日志文件
fh = logging.FileHandler('app.log')
fh.setLevel(logging.DEBUG)

# 创建控制台处理器，输出到控制台
ch = logging.StreamHandler()

# 设置日志格式
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)

# 将处理器添加到logger中
# 创建控制台处理器，输出到控制台
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
ch.setFormatter(formatter)

if not any(isinstance(handler, logging.StreamHandler) for handler in logger.handlers):
    logger.addHandler(fh)
    logger.addHandler(ch)
    print("StreamHandler added")
else:
    print("StreamHandler already exists")


USE_TOY_DATASET=False

StreamHandler added


# Helper functions

In [3]:
def load_dataset(X_path,y_path,X_domain_path=None,do_standardisation=False,test_size=0.1,random_state=42):
    X_df=None
    X_domain_info=None
    y_df=None
    with open(X_path, 'rb') as f:
        X_df = pickle.load(f)
    with open(y_path, 'rb') as f:
        y_df = pickle.load(f)
    print(X_df.shape)
    print(y_df.shape)
    if X_domain_path!=None:
        with open(X_domain_path, 'rb') as f:
            X_domain_info = pickle.load(f)
        print(X_domain_info.shape)
        X_train, X_test, y_train, y_test,X_D_train,X_D_test = train_test_split(X_df, y_df,X_domain_info, test_size=test_size, random_state=random_state)
    else:
        X_train, X_test, y_train, y_test= train_test_split(X_df, y_df, test_size=test_size, random_state=random_state)
    X_train = np.array(X_train, dtype=np.float32)
    y_train = np.array(y_train, dtype=np.float32)

    if X_domain_path!=None:
        X_D_train = np.array(X_D_train, dtype=np.float32)
        X_D_test = np.array(X_D_test, dtype=np.float32)

    X_test = np.array(X_test, dtype=np.float32)
    y_test = np.array(y_test, dtype=np.float32)
    if do_standardisation:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

    X_train_tensor = torch.tensor(X_train)
    y_train_tensor = torch.tensor(y_train)
    if X_domain_path!=None:
        X_D_train_tensor = torch.tensor(X_D_train)
        X_D_test_tensor = torch.tensor(X_D_test)

    X_test_tensor = torch.tensor(X_test)
    y_test_tensor = torch.tensor(y_test)

    if X_domain_path!=None: 
        return (X_train_tensor,X_D_train_tensor,y_train_tensor),(X_test_tensor,X_D_test_tensor,y_test_tensor)
    else:
        return (X_train_tensor,y_train_tensor),(X_test_tensor,y_test_tensor)

def sigmoid_4_param(x, x0, L, k, d):
    """ Comparing with Dennis Wang's sigmoid:
    x0 -  p - position, correlation with IC50 or EC50
        bounds [0, 1]
    L = 1 in Dennis Wang's sigmoid, protect from devision by zero if x is too small 
        L<1 inverted sigmoid, l=100 - lower upper and lower boundso sigmpoid on y axis (y= [0.1, 0.11])
        bounds [0.8, 10]
    k = -1/s (s -shape parameter)  default = -10 k=0 straight line, k<0 sigmoid around k=-10
        bounds [1, -100]
    d - determines the vertical position of the sigmoid - shift on y axis - better fitting then Dennis Wang's sigmoid
         bounds [0, 0.9]
    parameters_bound ((0, 0.8, -100, 0), (1, 10, 1, 0.9))
    """
    return ( 1/ (L + np.exp(-k*(x-x0))) + d)

def overwrite_to_test():
    VAR=0.09
    AMP=1.0
    train_size=100

    X_sin=torch.linspace(0, 1, train_size)
    X_cos=torch.linspace(0, 1, train_size)
    X_sig=torch.linspace(0, 1, train_size)
    y_sin = AMP*torch.sin(X_sin * (2 * math.pi)) + torch.randn(X_sin.size()) * math.sqrt(VAR)
    y_cos = AMP*torch.cos(X_cos * (2 * math.pi)) + torch.randn(X_cos.size()) * math.sqrt(VAR)
    y_sig = AMP*(torch.sigmoid(15*(X_sig-0.5))-0.5)*2+torch.randn(X_cos.size()) * math.sqrt(VAR)
    # Training data is 100 points in [0,1] inclusive regularly spaced
    X_train_tensor = torch.linspace(0, 1, train_size)
    # True function is sin(2*pi*x) with Gaussian noise

    num_conc=1
    num_feat=1
    nums_domain=torch.Tensor([3])
    num_data=train_size

    X_sin = X_sin.unsqueeze(1)
    X_cos = X_cos.unsqueeze(1)
    X_sig = X_sig.unsqueeze(1)
    y_sin = y_sin.unsqueeze(1)
    y_cos = y_cos.unsqueeze(1)
    y_sig = y_sig.unsqueeze(1)

    X_sin_domain=torch.zeros(train_size,1)
    X_cos_domain=torch.ones(train_size,1)
    X_sig_domain=torch.ones(train_size,1)*2
    X_sin_cat = torch.cat((X_sin_domain, X_sin), dim=1)
    X_cos_cat = torch.cat((X_cos_domain, X_cos), dim=1)
    X_sig_cat = torch.cat((X_sig_domain, X_sig), dim=1)
    X_train_tensor = torch.cat((X_sin_cat, X_cos_cat,X_sig_cat), dim=0)
    y_train_tensor = torch.cat((y_sin, y_cos,y_sig), dim=0)

    f, ax = plt.subplots(1, 3, figsize=(12, 3))
    for i in range(3):
        ax[i].plot(X_train_tensor[i*train_size:(i+1)*train_size,1:].squeeze().numpy(),y_train_tensor[i*train_size:(i+1)*train_size], 'k*')
        axis=X_train_tensor[i*train_size:(i+1)*train_size,1:].flatten().numpy()
        ax[i].set_ylim([-3, 3])
        ax[i].legend(['Observed Data', 'Mean', 'Confidence'])

    # indices = torch.randperm(X_train_tensor.size(0))
    # X_train_tensor = X_train_tensor[indices]
    # y_train_tensor = y_train_tensor[indices]

    test_size=50
    X_sin=torch.linspace(0, 1, test_size)
    X_cos=torch.linspace(0, 1, test_size)
    X_sig=torch.linspace(0, 1, test_size)

    X_sin_domain=torch.zeros(test_size,1)
    X_cos_domain=torch.ones(test_size,1)
    X_sig_domain=torch.ones(test_size,1)*2

    X_sin = X_sin.unsqueeze(1)
    X_cos = X_cos.unsqueeze(1)
    X_sig = X_sig.unsqueeze(1)

    y_sin =AMP* torch.sin(X_sin * (2 * math.pi)) + torch.randn(X_sin.size()) * math.sqrt(VAR)
    y_cos =AMP* torch.cos(X_cos * (2 * math.pi)) + torch.randn(X_cos.size()) * math.sqrt(VAR)
    y_sig =AMP* (torch.sigmoid(15*(X_sig-0.5))-0.5)*2+torch.randn(X_cos.size()) * math.sqrt(VAR)

    X_sin_cat = torch.cat((X_sin_domain, X_sin), dim=1)
    X_cos_cat = torch.cat((X_cos_domain, X_cos), dim=1)
    X_sig_cat = torch.cat((X_sig_domain, X_sig), dim=1)

    # X_test_tensor =X_sin_cat
    # y_test_tensor =y_sin
    
    X_test_tensor = torch.cat((X_sin_cat, X_cos_cat,X_sig_cat), dim=0)
    y_test_tensor = torch.cat((y_sin, y_cos,y_sig), dim=0)

    # X_test_tensor = torch.cat((X_sin_cat, X_cos_cat), dim=0)
    # y_test_tensor = torch.cat((y_sin, y_cos), dim=0)  





In [4]:
class config():
    NUMS_DOMAIN=None
    NUMS_DOMAIN_FEATURE=None
    NUMS_DOMAIN_AS_INT=None
    NUM_CONC=None
    STEP_SIZE=None
    lr=None
    gamma=None
    def __init__(self, NUMS_DOMAIN=None,
                 NUMS_DOMAIN_FEATURE=None,
                 NUMS_DOMAIN_AS_INT=None,
                 NUM_CONC=None,
                 STEP_SIZE=None,
                 lr=None,
                 gamma=None):
        self.NUMS_DOMAIN=NUMS_DOMAIN
        self.NUMS_DOMAIN_FEATURE=NUMS_DOMAIN_FEATURE
        self.NUMS_DOMAIN_AS_INT=NUMS_DOMAIN_AS_INT
        self.NUM_CONC=NUM_CONC
        self.STEP_SIZE=STEP_SIZE
        self.lr=lr
        self.gamma=gamma


## run_test

In [5]:
def run_test(X_train_tensor,y_train_tensor,X_test_tensor,y_test_tensor,model,kernel,config):
    likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=config.NUM_CONC)

    m = model(X_train_tensor, y_train_tensor, likelihood,kernel,config)

    training_iterations = 500

    # Find optimal model hyperparameters
    m.train()
    likelihood.train()
    
    optimizer = torch.optim.Adam(m.parameters(), lr=config.lr)
    STEP_SIZE=config.STEP_SIZE
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=config.gamma)
    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, m)
    last_loss=1
    avg_loss=0
    this_loss=0
    for i in range(training_iterations):
        try:
            optimizer.zero_grad()
            output = m(X_train_tensor)
            loss = -mll(output, y_train_tensor)
            loss.backward()
                
            this_loss=loss.item()
            avg_loss+=this_loss
            optimizer.step()  # 更新参数
            scheduler.step()  # 更新学习率
            
            if i%STEP_SIZE==STEP_SIZE-1:
                logger.debug('Iter %d/%d - Loss: %.3f' % (i + 1, training_iterations, loss.item()))
                avg_loss=avg_loss/STEP_SIZE
                if abs((this_loss-avg_loss)/avg_loss)<0.001:
                    break
                        
                avg_loss=0
                    
        except Exception as e:
            logger.error(f"发生了一个异常: {e}")
            continue
        
    m.eval()
    likelihood.eval()
    # Make predictions
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        distribution = likelihood(m(X_test_tensor))
        mean = distribution.mean
        lower, upper = distribution.confidence_region()
    nll = -torch.distributions.Normal(mean, distribution.variance.sqrt()).log_prob(y_test_tensor).mean().item()

    rmse=torch.sqrt(torch.tensor(mean_squared_error(y_test_tensor.numpy(), mean.numpy()))).item() 
    nmse = rmse / torch.var(y_test_tensor).item()

    logger.info(f'NLL: {nll:.4f}')
    logger.info(f'RMSE: {rmse:.4f}')
    logger.info(f'NMSE: {nmse:.4f}')




# Kernels

## Kernel Exponential Squared

In [6]:
class Kernel_Exponential_Squared(Kernel):
    def __init__(self,config, **kwargs):
        super().__init__(**kwargs)
        self.height_scales=torch.nn.Parameter(torch.ones((config.NUMS_DOMAIN)))
        self.length_scales=torch.nn.Parameter(torch.ones((config.NUMS_DOMAIN))) 
        self.register_parameter(name='height_scales', parameter=self.height_scales)
        self.register_parameter(name='length_scales', parameter=self.length_scales)
        self.config=config
    def forward(self, x1, x2, diag=False, last_dim_is_batch=False, **params):
        # x1_domain   =   x1[:, :num_domain_feat]
        # x1_genetics =   x1[:, num_domain_feat:]
        # x2_domain   =   x2[:, :num_domain_feat]
        # x2_genetics =   x2[:, num_domain_feat:]

        x1_domain   =   x1[:, :self.config.NUMS_DOMAIN_FEATURE]
        x1_genetics =   x1
        x2_domain   =   x2[:, :self.config.NUMS_DOMAIN_FEATURE]
        x2_genetics =   x2

        dist_mat=self.covar_dist(x1_genetics, x2_genetics, square_dist=True, diag=diag, **params)

        height_scales_parsed_1 = self.height_scales[x1_domain.long()].flatten()
        height_scales_parsed_2 = self.height_scales[x2_domain.long()].flatten()
        length_scales_parsed_1 = self.length_scales[x1_domain.long()].flatten()
        length_scales_parsed_2 = self.length_scales[x2_domain.long()].flatten()

        part_L1L2T=torch.sqrt(torch.outer(length_scales_parsed_1*length_scales_parsed_1,length_scales_parsed_2.T*length_scales_parsed_2.T))
        part_L1L1T=length_scales_parsed_1*length_scales_parsed_1
        part_L2L2T=length_scales_parsed_2*length_scales_parsed_2
        part_L1sqrL2sqr=torch.outer(part_L1L1T,torch.ones_like(part_L2L2T).T)+torch.outer(torch.ones_like(part_L1L1T),part_L2L2T)

        part_1=torch.outer(height_scales_parsed_1,height_scales_parsed_2.T)
        part_2=torch.sqrt(2*part_L1L2T/part_L1sqrL2sqr)
        part_3=torch.exp(-dist_mat/part_L1sqrL2sqr)

        result=part_1*part_2*part_3
        if diag:
            return result.diag()
        return result

## K_MS

In [7]:

    
class K_MS(Kernel):
    def __init__(self,config, **kwargs):
        super().__init__(**kwargs)
        self.domain_coefficient=torch.nn.Parameter(torch.rand((config.NUMS_DOMAIN))*2-1)
        self.K_ES=Kernel_Exponential_Squared(config)
        self.register_parameter(name='domain_coefficient', parameter=self.domain_coefficient)
        self.config=config
    def forward(self, x1, x2, diag=False, last_dim_is_batch=False, **params):
        x1_domain   =   x1[:, :self.config.NUMS_DOMAIN_FEATURE]
        x2_domain   =   x2[:, :self.config.NUMS_DOMAIN_FEATURE]

        domain_coefficient_parsed_1=self.domain_coefficient[x1_domain.long()].flatten()
        domain_coefficient_parsed_2=self.domain_coefficient[x2_domain.long()].flatten()
        domain_scaler=torch.outer(torch.tanh(domain_coefficient_parsed_1),torch.tanh(domain_coefficient_parsed_2.T))

        domain_mat=torch.outer(x1_domain.flatten()+1,1/(x2_domain.flatten().T+1))
        mask = (domain_mat == 1.0)

        # 等于1的位置置0
        domain_mat[mask] = 0.0

        # 不等于1的位置置1
        domain_mat[~mask] = 1.0
        # print(domain_scaler)

        
        base_cov=self.K_ES(x1,x2,diag=diag, last_dim_is_batch=last_dim_is_batch, **params)

        final_scaler=(domain_scaler-1.0)*domain_mat+1.0
        return base_cov*final_scaler


## K_Alpha_Beta

In [8]:
class K_Alpha_Beta(Kernel):
    def __init__(self,config, **kwargs):
        super().__init__(**kwargs)
        self.kernels=[]
        for _ in range(config.NUMS_DOMAIN):
            self.kernels.append(RBFKernel())

        self.alpha= torch.nn.Parameter(torch.ones(config.NUMS_DOMAIN))
        self.beta=torch.nn.Parameter(torch.ones(config.NUMS_DOMAIN))
    def forward(self, x1, x2, diag=False, last_dim_is_batch=False, **params):#
        x1_domain   =   x1[:, :config.NUMS_DOMAIN_FEATURE]
        x2_domain   =   x2[:, :config.NUMS_DOMAIN_FEATURE]
        x1_num_data=x1_domain.shape[0]
        x2_num_data=x2_domain.shape[0]
        cov_mats_alpha=torch.zeros([x1_num_data,x2_num_data])
        cov_mats_beta=torch.zeros([x1_num_data,x2_num_data])

        alpha_reparameterized=torch.sqrt(self.alpha*self.alpha)
        beta_reparameterized=torch.sqrt(self.beta*self.beta)
        for i in range(config.NUMS_DOMAIN):
            cov_mats_alpha=cov_mats_alpha.add(self.kernels[i](x1, x2, diag=diag, last_dim_is_batch=last_dim_is_batch, **params)*alpha_reparameterized[i])
            cov_mats_beta=cov_mats_beta.add(self.kernels[i](x1, x2, diag=diag, last_dim_is_batch=last_dim_is_batch, **params)*beta_reparameterized[i])

        domain_mat_alpha=torch.outer(x1_domain.flatten()+1,1/(x2_domain.flatten().T+1))
        domain_mat_beta=torch.outer(x1_domain.flatten()+1,1/(x2_domain.flatten().T+1))
        mask = (domain_mat_alpha == 1.0)
        domain_mat_alpha[mask] = 0.0
        domain_mat_alpha[~mask] = 1.0
        domain_mat_beta[mask] = 1.0
        domain_mat_beta[~mask] = 0.0

        
        result=cov_mats_alpha*domain_mat_alpha+cov_mats_beta*domain_mat_beta
        if diag:
            return result.diag()
        return result

## Transferable quadriple

In [9]:
class Transferable_Quadriple_Kernel(Kernel):
    def __init__(self,config, **kwargs):
        super().__init__(**kwargs)
        # self.kernel_mu = gpytorch.kernels.RBFKernel()
        # self.kernel_met = gpytorch.kernels.RBFKernel()
        # self.kernel_cn = gpytorch.kernels.RBFKernel()
        # self.kernel_dc = gpytorch.kernels.RBFKernel()
        # self.filter_mu,self.filter_met,self.filter_cn,self.filter_dc=filters
        self.temp=RBFKernel()
        initial_value = torch.ones(config.NUMS_DOMAIN)
        self.domain_relateness = torch.nn.Parameter(initial_value)

        # 使用 softplus 变换来确保参数为正值
        self.register_constraint("domain_relateness", gpytorch.constraints.Positive())
        
    def forward(self, x1, x2, diag=False, last_dim_is_batch=False, **params):
        x1_domain   =   x1[:, :self.config.NUMS_DOMAIN_FEATURE]
        x1_genetics =   x1[:, self.config.NUMS_DOMAIN_FEATURE:]
        x2_domain   =   x2[:, :self.config.NUMS_DOMAIN_FEATURE]
        x2_genetics =   x2[:, self.config.NUMS_DOMAIN_FEATURE:]
        # # Apply the base kernel
        # filter_mu=self.filter_mu
        # filter_met=self.filter_met
        # filter_cn=self.filter_cn
        # filter_dc=self.filter_dc
        
        # x1_mu   =   x1_genetics @ filter_mu
        # x1_met  =   x1_genetics @ filter_met
        # x1_cn   =   x1_genetics @ filter_cn
        # x1_dc   =   x1_genetics @ filter_dc

        # x2_mu   =   x2_genetics @ filter_mu
        # x2_met  =   x2_genetics @ filter_met
        # x2_cn   =   x2_genetics @ filter_cn
        # x2_dc   =   x2_genetics @ filter_dc

        domain_relateness_multiplier=self.domain_relateness
        # 获取领域系数
        domain_factors1 = self.domain_relateness[x1_domain.long()]
        domain_factors2 = self.domain_relateness[x2_domain.long()]

        # base_cov=self.kernel_mu(x1_mu,x2_mu)*self.kernel_met(x1_met,x2_met)*self.kernel_cn(x1_cn,x2_cn)* 1#self.kernel_dc(x1_dc,x2_dc)
        base_cov=self.temp(x1_genetics,x2_genetics)
        # print(f"Base kernel:{base_cov.shape}")
        if diag:
            adjusted_cov = base_cov * (domain_factors1 * domain_factors2).squeeze()
            # print(f"Diag Adjusted kernel:{adjusted_cov.shape}")
        else:
            factor= (domain_factors1 @ domain_factors2.T)
            adjusted_cov = base_cov *factor
            # print(f"Non-diag Adjusted kernel:{adjusted_cov.shape}")
            # print(f"factor:{factor.shape}")
            # print(f"domain_factors1:{domain_factors1.shape}")
            # print(f"domain_factors2:{domain_factors2.shape}")
        return adjusted_cov


# Models

## Linear model of corregionalization

In [10]:
class Linear_Model_Of_Corregionalization(gpytorch.models.ExactGP):
    #TODO add input for domain information
    def __init__(self, train_x, train_y,likelihood,kernel,config):
        super(Linear_Model_Of_Corregionalization, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ConstantMean(), num_tasks=config.NUM_CONC
        )
        kernels=[]
        if kernel==RBFKernel:
            for _ in range(config.NUM_CONC):
                kernels.append(kernel())
        else:
            for _ in range(config.NUM_CONC):
                kernels.append(kernel(config))
        self.covar_module =gpytorch.kernels.LCMKernel(
            kernels, num_tasks=config.NUM_CONC, rank=config.NUM_CONC
        )

        logger.debug("Completed")

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)

## Multitask GP

In [11]:
class MultitaskGP(gpytorch.models.ExactGP):
    #TODO add input for domain information
    def __init__(self, train_x, train_y,likelihood,kernel,config):
        super(MultitaskGP, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ConstantMean(), num_tasks=config.NUM_CONC
        )
        kernels=[]
        
        if kernel==RBFKernel:
            kern=kernel()
        else:
            kern=kernel(config)
                
        self.covar_module =gpytorch.kernels.MultitaskKernel(
            kern, num_tasks=config.NUM_CONC, rank=config.NUM_CONC
        )

        
        if USE_TOY_DATASET:
            self.covar_module = gpytorch.kernels.MultitaskKernel(
                [RBFKernel(),  # 2 latent GP's
             RBFKernel(),RBFKernel()], num_tasks=config.NUM_CONC,rank=config.NUM_CONC
            )
        #     self.covar_module = gpytorch.kernels.MultitaskKernel(
        #         K_MS(),  # 2 latent GP's

        #     num_tasks=num_conc, rank=num_conc-1
        # )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)

In [12]:
# X_path="data/X_df_toy.pkl"
# y_path="data/y_df_toy.pkl"

# X_path="data/X_df_Shikonin.pkl"
# y_path="data/y_df_Shikonin.pkl"
# X_domain_path="data/X_domain_info_Shikonin.pkl"

Global=config()

X_path="data/X_df_2_shots.pkl"
y_path="data/y_df_2_shots.pkl"
X_domain_path="data/X_domain_info_2_shots.pkl"

# X_path="data/2_shots_4_params_sigmoid/X_df.pkl"
# y_path="data/2_shots_4_params_sigmoid/y_df.pkl"
# X_domain_path="data/2_shots_4_params_sigmoid/X_domain_info.pkl"
(X_train_tensor,X_D_train_tensor,y_train_tensor),(X_test_tensor,X_D_test_tensor,y_test_tensor)=load_dataset(X_path,y_path,do_standardisation=True,X_domain_path=X_domain_path)

print(X_train_tensor.shape)
print(X_D_train_tensor.shape)
print(y_train_tensor.shape)

Global.NUM_CONC=y_train_tensor.shape[1]

Global.NUM_FEAT=X_train_tensor.shape[1]
Global.NUM_DOMAIN_FEAT=X_D_train_tensor.shape[1]
NUMS_DOMAIN, max_indices_row = torch.max(X_D_train_tensor, dim=0)
NUMS_DOMAIN.add_(1)


print(NUMS_DOMAIN.item())
Global.NUMS_DOMAIN=NUMS_DOMAIN.long()
X_train_tensor = torch.cat((X_D_train_tensor, X_train_tensor), dim=1)
X_test_tensor = torch.cat((X_D_test_tensor, X_test_tensor), dim=1)
print(X_train_tensor.shape)
print(X_test_tensor.shape)

with open(X_path, 'rb') as f:
    X_df = pickle.load(f)
mask_bool_met = X_df.columns.str.contains("HypMET")
mask_bool_mut = X_df.columns.str.contains("mut")
mask_bool_cna = X_df.columns.str.contains("cna")
mask_bool_dc  = X_df.columns.str.contains("dc")

mask_float_met = np.array(mask_bool_met, dtype=np.float32) 
mask_float_mut = np.array(mask_bool_mut, dtype=np.float32)
mask_float_cna = np.array(mask_bool_cna, dtype=np.float32)
mask_float_dc  = np.array(mask_bool_dc, dtype=np.float32)

diag_matrix_met = torch.tensor(np.diag(mask_float_met))
diag_matrix_mut = torch.tensor(np.diag(mask_float_mut))
diag_matrix_cna = torch.tensor(np.diag(mask_float_cna))
diag_matrix_dc  = torch.tensor(np.diag(mask_float_dc))
filters=(diag_matrix_mut,diag_matrix_met,diag_matrix_cna,diag_matrix_dc)
print("MUT features:"+str(np.sum(mask_float_mut))+str(diag_matrix_met.shape))
print("Met features:"+str(np.sum(mask_float_met))+str(diag_matrix_met.shape))
print("CNA features:"+str(np.sum(mask_float_cna))+str(diag_matrix_met.shape))
print("DC features:"+str(np.sum(mask_float_dc))+str(diag_matrix_met.shape))  

(62, 1084)
(62, 10)
(62, 1)
torch.Size([55, 1084])
torch.Size([55, 1])
torch.Size([55, 10])
13.0
torch.Size([55, 1085])
torch.Size([7, 1085])
MUT features:310.0torch.Size([1084, 1084])
Met features:338.0torch.Size([1084, 1084])
CNA features:425.0torch.Size([1084, 1084])
DC features:0.0torch.Size([1084, 1084])


# Tests

In [13]:
Global.lr=0.1
Global.gamma=0.5
Global.STEP_SIZE=50
Global.NUMS_DOMAIN_FEATURE=1
print(Global.NUMS_DOMAIN[0].item())

13


In [14]:

models=[MultitaskGP,Linear_Model_Of_Corregionalization]
kernels=[K_MS,K_Alpha_Beta,RBFKernel]

run_test(X_train_tensor,y_train_tensor,X_test_tensor,y_test_tensor,model=Linear_Model_Of_Corregionalization,kernel=K_MS,config=Global)

2024-08-12 19:35:42,958 - DEBUG - Completed
C:\Users\25858\AppData\Local\Temp\ipykernel_11444\3416435673.py:14: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at C:\b\abs_8f7uhuge1i\croot\pytorch-select_1717607507421\work\aten\src\ATen\native\TensorShape.cpp:3679.)
  domain_scaler=torch.outer(torch.tanh(domain_coefficient_parsed_1),torch.tanh(domain_coefficient_parsed_2.T))
2024-08-12 19:35:46,643 - DEBUG - Iter 50/500 - Loss: -0.646
2024-08-12 19:35:49,329 - DEBUG - Iter 100/500 - Loss: -1.095
2024-08-12 19:35:51,974 - DEBUG - Iter 150/500 - Loss: -1.210
2024-08-12 19:35:54,754 - DEBUG - Iter 200/500 - Loss: -1.244
2024-08-12 19:35:57,527 - DEBUG - Iter 250/500 - Loss: -1.257
2024-08-12 19:36:00,151 - DEBUG - Iter 300/500 - 

In [15]:
if USE_TOY_DATASET == True:
    # Get into evaluation (predictive posterior) mode
    model.eval()
    likelihood.eval()

    # Test points are regularly spaced along [0,1]
    # Make predictions by feeding model through likelihood
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        observed_pred = likelihood(model(X_test_tensor))

    with torch.no_grad():
        # Initialize plot
        f, ax = plt.subplots(1, 3, figsize=(12, 3))

        lower, upper = observed_pred.confidence_region()
        mean=observed_pred.mean.numpy()
        for i in range(3):
            ax[i].plot(X_test_tensor[i*50:(i+1)*50,1:].squeeze().numpy(),mean[i*50:(i+1)*50], 'k*')
            ax[i].fill_between(X_test_tensor[i*50:(i+1)*50,1:].flatten().numpy(), lower[i*50:(i+1)*50].squeeze().numpy(), upper[i*50:(i+1)*50].squeeze().numpy(), alpha=0.5)
            ax[i].set_ylim([-3, 3])
            ax[i].legend(['Observed Data', 'Mean', 'Confidence'])
